## Desk allocation problem
### IQVIA homework
#### by Simon Zatko
---

### imports, constants, files

In [1]:
import csv
import random
from difflib import SequenceMatcher

In [2]:
CORR = 1    # corridor/obstacle in office 2d array
FREE = 0    # free desk in office 2d array
SIM_TEAM_RATE = 0.4   # hyperparameter - which team similarity we consider as 1 group

In [3]:
with open('Employees.csv', newline='') as f:
    reader = csv.reader(f)
    employees = list(reader)

### help functions

In [4]:
def get_office_size(office):    # get office height and width
    return len(office), len(office[0])

def get_similarity(str1, str2):    # get 2 strings similarity - needed for team similarity
    return SequenceMatcher(None, str1, str2).ratio()

def get_employee_dic(employees):    # convert list of employees to dictionary
    emp_dic = {}
    for emp in employees:
        emp_dic[emp[0]] = emp[1]
    return emp_dic

def get_neighbors_count(office, i, j):    # get count of employees in nearest desks
    p, q = get_office_size(office)
    neighbors = 0
    if i > 0 and office[i-1][j] not in (FREE, CORR):
        neighbors += 1 
    if j > 0 and office[i][j-1] not in (FREE, CORR):
        neighbors += 1 
    if i < p-1 and office[i+1][j] not in (FREE, CORR):
        neighbors += 1 
    if j < q-1 and office[i][j+1] not in (FREE, CORR):
        neighbors += 1 
    return neighbors

def get_free_neighbors_count(office, i, j):    # get count of free desks in nearest desks
    p, q = get_office_size(office)
    neighbors = 0
    if i > 0 and office[i-1][j] == FREE:
        neighbors += 1 
    if j > 0 and office[i][j-1] == FREE:
        neighbors += 1 
    if i < p-1 and office[i+1][j] == FREE:
        neighbors += 1 
    if j < q-1 and office[i][j+1] == FREE:
        neighbors += 1 
    return neighbors

def get_first_free_neighbor_coords(office, i, j):    # get first nearest free desk positions
    p, q = get_office_size(office)
    if i > 0 and office[i-1][j] == FREE:
        return i-1,j
    if j > 0 and office[i][j-1] == FREE:
        return i,j-1 
    if i < p-1 and office[i+1][j] == FREE:
        return i+1,j 
    if j < q-1 and office[i][j+1] == FREE:
        return i,j+1
    return False

### 1. random_office
- this function generate pseudo-random 2D array of free desks and other items:
    - we create corridors, based on density of desks in PxQ array
    - we add random items/obstacles to adjust D - number of free desks

In [5]:
def random_office(p, q, d):
    matrix = [[FREE]*q for i in range(p)]
    dens = int(d / (p*q) * 10)
    if dens < 3:
        dens = 3
    max_corrs = p*q - d

    for i in range(p):
        for j in range(q):
            if max_corrs and (i%dens == dens-1 or j%dens == dens-1):
                matrix[i][j] = CORR
                max_corrs -= 1

    while max_corrs > 0:
        rand_p = random.randint(0,p-1)
        rand_q = random.randint(0,q-1)
        if matrix[rand_p][rand_q] == FREE:
            matrix[rand_p][rand_q] = CORR
            max_corrs -= 1

    return matrix

### 2. collocation_score
- this functions returns value (0,1), which represent how good are employees seated in office
- best score is achieved by allocation some team together, but we have to consider also team / groups similarities (2 developers are better than dev + analyst)

In [6]:
def get_neighbors_best_similarity_score(office, i, j, employees, emp_id):
    p, q = get_office_size(office)
    emp_dic = get_employee_dic(employees)
    emp_team = emp_dic[emp_id]
    best_similarity = 0

    nbrs_cnt = get_neighbors_count(office, i, j)
    # take the same / most similar team names from employees in office
    best_nbrs = list({k: v for k, v in sorted(emp_dic.items(), key=lambda item: get_similarity(item[1], emp_team), reverse=True)[:nbrs_cnt]}.values())
    for nbr in best_nbrs:
        best_similarity += get_similarity(emp_team, nbr)
    
    return best_similarity
    

def get_neighbors_curr_similarity_score(office, i, j, employees, emp_id):
    p, q = get_office_size(office)
    emp_dic = get_employee_dic(employees)
    emp_team = emp_dic[emp_id]
    curr_similarity = 0
    
    if i > 0 and office[i-1][j] not in (FREE, CORR):
        curr_similarity += get_similarity(emp_team, emp_dic[office[i-1][j]])
    if j > 0 and office[i][j-1] not in (FREE, CORR):
        curr_similarity += get_similarity(emp_team, emp_dic[office[i][j-1]])
    if i < p-1 and office[i+1][j] not in (FREE, CORR):
        curr_similarity += get_similarity(emp_team, emp_dic[office[i+1][j]])
    if j < q-1 and office[i][j+1] not in (FREE, CORR):
        curr_similarity += get_similarity(emp_team, emp_dic[office[i][j+1]])
    
    return curr_similarity
    

def collocation_score(office, employees):
    p, q = get_office_size(office)
    best_similarity = 0
    curr_similarity = 0
    for i in range(p):
        for j in range(q):
            if office[i][j] not in (FREE, CORR):
                best_similarity += get_neighbors_best_similarity_score(office, i, j, employees, office[i][j])
                curr_similarity += get_neighbors_curr_similarity_score(office, i, j, employees, office[i][j])              
                
    if best_similarity:
        return curr_similarity / best_similarity
    return 0

### 3. assign_desk
- assign desk to new employees sequentally:
    - take a seat near to colleque from the same team or eventually from similar team / team-group (developers, testers, analysts etc.)
    - if there are no similar colleque, take a random desk from office

In [7]:
def assign_emp_to_desk(office, i, j, emp, seated):
    office[i][j] = emp[0]
    free_neighbors = get_neighbors_count(office, i, j)
    if free_neighbors:
        seated.append([emp[1], i, j])

def assign_desk(office, employees):
    p, q = get_office_size(office)
    seated = []
    
    for emp in employees:
        most_similar_value = 0
        most_similar = False
        
        for seat in seated:
            similarity = SequenceMatcher(None, seat[0], emp[1]).ratio()
            if similarity >= SIM_TEAM_RATE and similarity > most_similar_value and get_free_neighbors_count(office, seat[1], seat[2]):
                most_similar = seat
              
        if most_similar:
            new_i, new_j = get_first_free_neighbor_coords(office, most_similar[1], most_similar[2])
            assign_emp_to_desk(office, new_i, new_j, emp, seated)
        else:
            while 1:
                i = random.randint(0,p-1)
                j = random.randint(0,q-1)
                if office[i][j] == 0:
                    assign_emp_to_desk(office, i, j, emp, seated)
                    break

### 4. evaluation

In [8]:
office = random_office(10, 10, 60)
employees_sub = random.sample(employees[1:], 50)
assign_desk(office, employees_sub)
collocation_score(office, employees_sub)

0.5827870648032916

In [9]:
office = random_office(20, 20, 300)
employees_sub = random.sample(employees[1:], 300)
assign_desk(office, employees_sub)
collocation_score(office, employees_sub)

0.6030730615270523

In [10]:
office = random_office(15, 55, 500)
employees_sub = random.sample(employees[1:], 490)
assign_desk(office, employees_sub)
collocation_score(office, employees_sub)

0.5683930128404685